In [ ]:
import pandas as pd
import numpy as np  

# Read the data from the CSV file
data = pd.read_csv('allCommandWithTypeEmo.csv')

# Get the unique shop names
unique_shopname = data['ShopName'].unique()

# Convert the unique shop names to a DataFrame
unique_shopname_df = pd.DataFrame(unique_shopname, columns=['ShopName'])

# Output the unique shop names to a new CSV file
unique_shopname_df.to_csv('unique_shopnames.csv', index=False)

print("Unique shop names have been saved to 'unique_shopnames.csv'")

<class 'numpy.ndarray'>
['爾遇Dolce Vita(最新菜單請見臉書或IG)·開啟過的連結' '重慶故事金湯酸菜魚永華旗艦館' '夢東籬-永華店' '3皇3家-億載店'
 'Visions Bistro 微醺餐酒館安平創始店 / 亞果遊艇俱樂部景觀餐廳(台南景觀餐廳 台南餐酒館 漁光島美食 安平美食 安平餐酒館)'
 '吾亦私永華店' '弄 Relationship'
 'IMOMENT CAFÉ 享當下 台南旗艦店 （安平網美咖啡廳|熱門早午餐|網美早午餐|人氣餐廳|在地推薦餐廳|人氣早午餐|輕食下午茶）'
 '兜好麥鷄/土雞料理/安平美食/安平小吃/安平餐廳/安平restaurants/台南美食推薦/小甕土雞湯/桶子雞/鹽焗雞'
 '天塩川和室鍋物 ｜ 獨立包廂、親子友善餐廳' '光合箱子 台南店' '養蚵世家' '漁光島荳荳園' '逸隱軒人文茶坊'
 '米約廚房 Mi Yueh Kitchen(無接待七歲以下兒童）' '芙蓉食堂' '森島繪本咖啡屋|親子|輕食|鬆餅|繪本|親子餐廳|'
 'SIOU JHIH秀枝餐飲-永華店' '福豆屋安平旗艦店' '欣雅川菜館' '好好吃牛排館' '凱荳寵物家庭餐廳' '椿之味薏仁湯'
 '洁洁永華店' '滿玥軒' '波哥誠品安平店' '台南美食 牛園火鍋-安平店' '炸雞洋行 安平店' 'Beew Cafe 咖啡輕食餐廳'
 '慶平海產' '同記安平豆花(安平2店)' '漁光島元圓餐飲店(蚵仔煎停售）' '周氏蝦捲 老店' '寓･永恆'
 'Second Floor 貳樓台南店' '東牧景觀餐廳' '府城食府正宗台南料理-安平總店' '安平漁家' '丹丹漢堡 安平店'
 '老廣粵 花雕雞創意坊' '安平港海鮮餐廳' '椰庭餐廳（店休請看FB/IG)' 'JiNHER 今鶴 - 義式餐酒館台南安平店'
 '謝掌櫃虱目魚海鮮粥［安平總店］' '威尼斯Venice餐廳' '津昌貨櫃碼頭/安平熱炒/美食/附近美食/restaurants/吃的/必吃'
 'Woopen木盆 安平加盟門市' '橄饗家西班牙嚴選美食餐廳Como en casa gourmet' 'Jacky Bistro'
 '瑪莉洋房 義法料理' '雛菊餐桌-台南府前店' '居心地安平' '老城門 佰元鍋物 台南火鍋 安平美食'
 '天馬星

In [13]:
import torch 
classification = {"餐點": 0, "氣氛": 1, "兩者都有": 2}
emo_classification = {"正面": 1, "負面": 0}
shop_score_mean = data['ShopScore'].groupby(data['ShopName']).mean()

allShop = {}
current_shop = data['ShopName'][0]
need_arr = []
# One-hot encoding function for type with weights
def one_hot_encode_type(type_class):
    one_hot = np.zeros(len(classification))
    if type_class in classification:
        if type_class == "兩者都有":
            one_hot[classification[type_class]] = 1
        else:
            one_hot[classification[type_class]] = 0.5
    return one_hot

# One-hot encoding function for emo with weights based on type
def one_hot_encode_emo(emo_class, type_class):
    one_hot = np.zeros(len(emo_classification))
    weight = 1 if type_class == "兩者都有" else 0.5
    if emo_class in emo_classification:
        one_hot[emo_classification[emo_class]] = weight
    return one_hot

for idx, i in enumerate(data['ShopName']):
    if i != current_shop:
        allShop[current_shop] = np.mean(need_arr, axis=0) if need_arr else np.zeros(5)
        need_arr = []
        current_shop = i
    if not pd.isna(data.loc[idx, 'type']) and not pd.isna(data.loc[idx, 'emo']):
        type_class = data.loc[idx, 'type']
        emo_class = data.loc[idx, 'emo']
        one_hot_type = one_hot_encode_type(type_class)
        one_hot_emo = one_hot_encode_emo(emo_class, type_class)
        one_hot_vector = np.concatenate((one_hot_type, one_hot_emo))
        need_arr.append(one_hot_vector*shop_score_mean[i])
# Add the last shop's data
if need_arr:
    allShop[current_shop] = np.sum(need_arr, axis=0)
all_arrays = []
for shop in allShop.keys():
    # print(allShop[shop])
    all_arrays.append(allShop[shop])
    print(f"{shop}: {allShop[shop]}")
    print("=====================================")
# print(all_arrays)
# Stack all numpy arrays into a single numpy array
all_arrays_np = np.stack(all_arrays)
# Convert the numpy array to a PyTorch tensor
all_arrays_tensor = torch.tensor(all_arrays_np)
torch.save(all_arrays_tensor, 'one_hot_vectors.pt')
a = torch.load('one_hot_vectors.pt')

爾遇Dolce Vita(最新菜單請見臉書或IG)·開啟過的連結: [1.22643172 0.07400881 2.19911894 0.40176211 3.09779736]
重慶故事金湯酸菜魚永華旗艦館: [0.89538153 0.04919679 3.01084337 0.41325301 3.54216867]
夢東籬-永華店: [0.49538462 0.10615385 3.39692308 0.15333333 3.84512821]
3皇3家-億載店: [0.54944444 0.39416667 2.41277778 0.77638889 2.58      ]
Visions Bistro 微醺餐酒館安平創始店 / 亞果遊艇俱樂部景觀餐廳(台南景觀餐廳 台南餐酒館 漁光島美食 安平美食 安平餐酒館): [0.29518717 0.33208556 3.34545455 0.49197861 3.48074866]
吾亦私永華店: [0.82213542 0.18151042 2.09270833 1.00364583 2.09270833]
弄 Relationship: [0.575  0.0575 3.335  0.46   3.5075]
IMOMENT CAFÉ 享當下 台南旗艦店 （安平網美咖啡廳|熱門早午餐|網美早午餐|人氣餐廳|在地推薦餐廳|人氣早午餐|輕食下午茶）: [0.19202335 0.17373541 3.96848249 0.76809339 3.56614786]
兜好麥鷄/土雞料理/安平美食/安平小吃/安平餐廳/安平restaurants/台南美食推薦/小甕土雞湯/桶子雞/鹽焗雞: [1.78628049 0.0320122  0.56341463 0.22408537 2.15762195]
天塩川和室鍋物 ｜ 獨立包廂、親子友善餐廳: [0.35       0.19782609 3.80434783 0.1673913  4.18478261]
光合箱子 台南店: [0.69 0.13 2.46 0.77 2.51]
養蚵世家: [1.5695  0.043   1.075   0.54825 2.13925]
漁光島荳荳園: [1.15851064 0.07021277 1.94255319 0.67